In [6]:
import findspark
from pyspark.sql import SparkSession

findspark.init()

from pyspark import SparkContext,SparkConf
SparkContext.setSystemProperty('spark.executor.memory', '2g')
# SparkContext("local", "App Name").memory.offHeap.enabled

spark = SparkSession.builder\
        .master('local')\
        .appName('SparkSQL')\
        .getOrCreate()
sc = spark.sparkContext
config = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])
sc.stop()
sc = SparkContext(conf=config)
# spark.conf.set("spark.executor.memory", "2g")

# spark.memory.offHeap.enabled
# spark.memory.offHeap.size

# spark.config("spark.memory.offHeap.enabled","true") \
#         .config("spark.memory.offHeap.size","10g")
spark = SparkSession.builder\
        .master('local')\
        .appName('SparkSQL')\
        .getOrCreate()
df = spark.read.option("delimiter", ",").option("header", True).csv('pm_clean.csv')


In [2]:
df.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name_th: string (nullable = true)
 |-- name_en: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- _c0: string (nullable = true)
 |-- time_aq: string (nullable = true)
 |-- temp: string (nullable = true)
 |-- humid: string (nullable = true)
 |-- pm25_corrected: string (nullable = true)
 |-- pm10_corrected: string (nullable = true)



In [7]:
from pyspark.sql.functions import col
from pyspark.sql.functions import *
cols = ['lat', 'long','temp','humid','pm25_corrected','pm10_corrected']
for c in cols:
    df = df.withColumn(c, col(c).cast('double'))

In [4]:
df.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name_th: string (nullable = true)
 |-- name_en: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- _c0: string (nullable = true)
 |-- time_aq: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- humid: double (nullable = true)
 |-- pm25_corrected: double (nullable = true)
 |-- pm10_corrected: double (nullable = true)



In [5]:
df.show(5)

+---------------+---+-------------------+--------------------+---------+----------+---+--------------------+----+-----+--------------+--------------+
|      device_id| id|            name_th|             name_en|      lat|      long|_c0|             time_aq|temp|humid|pm25_corrected|pm10_corrected|
+---------------+---+-------------------+--------------------+---------+----------+---+--------------------+----+-----+--------------+--------------+
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084|  2|2020-07-20 18:17:...|33.1| 67.9|          11.2|          23.1|
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084|  6|2020-07-20 18:22:...|33.1| 69.3|          10.4|          21.6|
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084| 10|2020-07-20 18:27:...|32.9| 70.5|           9.7|          20.9|
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084| 14|2020-07-20 18

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
# df3 = df.withColumn(
#             'new_date',
#                 F.to_date(
#                     F.unix_timestamp('time_aq', 'yyyy-MM-dd HH:mm:ss:SSS').cast('timestamp')))
# df3=df.withColumn("time_aq",to_timestamp("input_timestamp")).show(truncate=False)
df3=df.withColumn("new_time",to_timestamp("time_aq"))

In [17]:
df3.show(5)

+---------------+---+--------------------+--------------------+---------+----------+-------+--------------------+----+-----+--------------+--------------+--------------------+
|      device_id| id|             name_th|             name_en|      lat|      long|    _c0|             time_aq|temp|humid|pm25_corrected|pm10_corrected|            new_time|
+---------------+---+--------------------+--------------------+---------+----------+-------+--------------------+----+-----+--------------+--------------+--------------------+
|868333031252642| 82|ศูนย์ศึกษาการพัฒน...|Huai Hong Khrai R...| 18.87813|  99.21817|3593250|2021-06-29 08:48:...|31.6| 63.5|          10.4|          23.1|2021-06-29 08:48:...|
|868333031266329| 67|(4) อาคาร 11 โครง...|(4) Building 11 B...| 13.60095| 100.74824|3908067|2021-07-09 05:54:...|32.2| 69.0|          14.1|          29.1|2021-07-09 05:54:...|
|868333031254515| 77|ซอยเพชรเกษม 43 บางแค|Soi Phetkasem 43,...|13.708905| 100.41382|5714048|2021-09-01 03:24:...|29.2| 9

In [ ]:
df3.show(20)

In [9]:
df3.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name_th: string (nullable = true)
 |-- name_en: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- _c0: string (nullable = true)
 |-- time_aq: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- humid: double (nullable = true)
 |-- pm25_corrected: double (nullable = true)
 |-- pm10_corrected: double (nullable = true)
 |-- new_date: date (nullable = true)



In [10]:
# df3.filter(df3['new_date'] >= F.to_date('2020-07-20 18:35:00',F.unix_timestamp( 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))).show()

TypeError: Column is not iterable

In [ ]:
spark.sql("select to_date('2020-07-20 18:35:00','yyyy-MM-dd HH:mm:ss') date") \
     .show()

+----------+
|      date|
+----------+
|2020-07-20|
+----------+



In [12]:
from pyspark.sql.functions import *
df4=spark.createDataFrame([["2020-07-20 18:35:00.000"],["2020-07-20 18:40:00.000"]],["input"])
df5 = df4.withColumn(
            'new_date',
                F.to_date(
                    F.unix_timestamp('input', 'yyyy-MM-dd HH:mm:ss:SSS').cast('timestamp')))

In [19]:
df5=df4.withColumn("new",to_timestamp("input"))
df5.show()

+--------------------+-------------------+
|               input|                new|
+--------------------+-------------------+
|2020-07-20 18:35:...|2020-07-20 18:35:00|
|2020-07-20 18:40:...|2020-07-20 18:40:00|
+--------------------+-------------------+



In [14]:
df4.show(4)

+--------------------+
|               input|
+--------------------+
|2020-07-20 18:35:...|
|2020-07-20 18:40:...|
+--------------------+



In [22]:
df5.show(5)

+--------------------+-------------------+
|               input|                new|
+--------------------+-------------------+
|2020-07-20 18:35:...|2020-07-20 18:35:00|
|2020-07-20 18:40:...|2020-07-20 18:40:00|
+--------------------+-------------------+



In [21]:
df5.collect()[0]['new']

datetime.datetime(2020, 7, 20, 18, 35)

In [23]:
df3.filter((df3['new_time'] >= df5.collect()[0]['new']) & (df3['new_time'] < df5.collect()[1]['new'])).show()

+---------------+---+--------------------+--------------------+---------+----------+---+--------------------+----+-----+--------------+--------------+--------------------+
|      device_id| id|             name_th|             name_en|      lat|      long|_c0|             time_aq|temp|humid|pm25_corrected|pm10_corrected|            new_time|
+---------------+---+--------------------+--------------------+---------+----------+---+--------------------+----+-----+--------------+--------------+--------------------+
|868333030868281| 24| *****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084| 18|2020-07-20 18:37:...|32.8| 71.6|           8.9|          19.4|2020-07-20 18:37:...|
|868333030873646| 23|*****กรมประชาสัมพ...|Government Public...|13.783098|100.540368| 16|2020-07-20 18:37:...|31.4| 79.1|           9.7|          20.9|2020-07-20 18:37:...|
|   30AEA49CC204| 17|ตึก 3 คณะวิศวกรรม...|Building 3, Facul...|13.736983|100.533476| 17|2020-07-20 18:37:...|35.7| 62.0|           8.5|     

In [ ]:
from pyspark.sql.functions import *
df4.select(col("input"), 
    to_timestamp(col("input"), "MM-dd-yyyy HH mm ss SSS").alias("to_timestamp") 
  ).show(truncate=False)

In [6]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
dff = spark.createDataFrame(data=data2,schema=schema)
dff.printSchema()
dff.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [13]:
df3.show()

+---------------+---+-------------------+--------------------+---------+----------+---+--------------------+----+-----+--------------+--------------+--------+
|      device_id| id|            name_th|             name_en|      lat|      long|_c0|             time_aq|temp|humid|pm25_corrected|pm10_corrected|new_date|
+---------------+---+-------------------+--------------------+---------+----------+---+--------------------+----+-----+--------------+--------------+--------+
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084|  2|2020-07-20 18:17:...|33.1| 67.9|          11.2|          23.1|    null|
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084|  6|2020-07-20 18:22:...|33.1| 69.3|          10.4|          21.6|    null|
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Department of Wat...|13.785348|100.539084| 10|2020-07-20 18:27:...|32.9| 70.5|           9.7|          20.9|    null|
|868333030868281| 24|*****กรมทรัพยากรน้ำ|Depar

In [ ]:
from pyspark.ml.feature import VectorAssembler
data_customer.columns
assemble=VectorAssembler(inputCols=[
 'BALANCE',
 'BALANCE_FREQUENCY',
 'PURCHASES',
 'ONEOFF_PURCHASES',
 'INSTALLMENTS_PURCHASES',
 'CASH_ADVANCE',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES_FREQUENCY',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'CASH_ADVANCE_FREQUENCY',
 'CASH_ADVANCE_TRX',
 'PURCHASES_TRX',
 'CREDIT_LIMIT',
 'PAYMENTS',
 'MINIMUM_PAYMENTS',
 'PRC_FULL_PAYMENT',
 'TENURE'], outputCol='features')
assembled_data=assemble.transform(data_customer)
assembled_data.show(2)


In [ ]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)
data_scale_output.show(2)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):

    KMeans_algo=KMeans(featuresCol='standardized', k=i)

    KMeans_fit=KMeans_algo.fit(data_scale_output)

    output=KMeans_fit.transform(data_scale_output)



    score=evaluator.evaluate(output)

    silhouette_score.append(score)

    print("Silhouette Score:",score)

In [ ]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel(‘k’)
ax.set_ylabel(‘cost’)